In [ ]:
from keras.layers import Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
import pathlib
import shutil
from shutil import copyfile
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## for real data
shutil.rmtree('./data/train')
shutil.rmtree('./data/test')
pathlib.Path('./data/train').mkdir(parents=True, exist_ok=True)
pathlib.Path('./data/test').mkdir(parents=True, exist_ok=True)

classes = ['green', 'red', 'unidentified']

for j in np.arange(len(classes)):
    cls = classes[j]
    
    print('processing class ' + cls + '\n')

    pathlib.Path('./data/train/'+ cls).mkdir(parents=True, exist_ok=True)
    pathlib.Path('./data/test/' + cls).mkdir(parents=True, exist_ok=True)

    path = './data/real_training_data/' + cls + '/'
    files = os.listdir(path)
    train_indx = np.random.choice(len(files), np.int(np.floor(len(files) * 0.7)), replace = False)
    test_indx = list(set(np.arange(len(files))) - set(train_indx))
    print(len(train_indx))
    print(len(test_indx))
    print(len(files))

    for i in np.arange(len(train_indx)):
        indx = train_indx[i]
        file = files[indx]
        src = path + file
        dest = './data/train/' + cls + '/' + file
        copyfile(src, dest)
    for i in np.arange(len(test_indx)):
        indx = test_indx[i]
        file = files[indx]
        src = path + file
        dest = './data/test/' + cls + '/' + file
        copyfile(src, dest)    

In [ ]:
## for simulation data -- append to previous
classes = ['green', 'red', 'unidentified']

for j in np.arange(len(classes)):
    cls = classes[j]
    
    print('processing class ' + cls + '\n')

    path = './data/sim_data_capture/' + cls + '/'
    files = os.listdir(path)
    train_indx = np.random.choice(len(files), np.int(np.floor(len(files) * 0.7)), replace = False)
    test_indx = list(set(np.arange(len(files))) - set(train_indx))
    print(len(train_indx))
    print(len(test_indx))
    print(len(files))

    for i in np.arange(len(train_indx)):
        indx = train_indx[i]
        file = files[indx]
        src = path + file
        dest = './data/train/' + cls + '/' + 'sim_' + file
        copyfile(src, dest)
    for i in np.arange(len(test_indx)):
        indx = test_indx[i]
        file = files[indx]
        src = path + file
        dest = './data/test/' + cls + '/' + 'sim_' + file
        copyfile(src, dest)    

In [ ]:
import keras
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, ELU
from keras.layers import Input, Activation, Dropout, merge
from keras.layers import Cropping2D, Lambda
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential

def Inception(layer, inner_depth, out_depth, activation):

    conv_1x1_1 = Convolution2D(out_depth, 1, 1, border_mode = 'same')(layer)

    conv_3x3 = Convolution2D(inner_depth, 1, 1, border_mode = 'same', activation = 'relu')(layer)
    conv_3x3 = Convolution2D(out_depth, 3, 3, border_mode = 'same')(conv_3x3)

    conv_5x5 = Convolution2D(inner_depth, 1, 1, border_mode = 'same', activation = 'relu')(layer)
    conv_5x5 = Convolution2D(out_depth, 5, 5, border_mode = 'same')(conv_5x5)

    conv_1x1_4 = MaxPooling2D((3, 3), strides = (1, 1), border_mode = 'same')(layer)
    conv_1x1_4 = Convolution2D(out_depth, 1, 1, border_mode = 'same')(conv_1x1_4)

    output = merge([conv_1x1_1, conv_3x3, conv_5x5, conv_1x1_4], concat_axis = 1)

    output = Activation(activation)(output)

    return output


def toy2():
    
    activation = 'relu'
    input_shape = (64, 64, 3)
    
    inp = Input(shape = input_shape)
    net = Convolution2D(32, 3, 3, border_mode = 'same')(inp)
    net = Activation(activation)(net)
    net = Convolution2D(32, 3, 3, border_mode = 'same')(net)
    net = Activation(activation)(net)
    net = MaxPooling2D((2, 2), strides = (2, 2))(net)
    net = Dropout(0.8)(net) 
   
    net = Inception(net, 32, 16, activation)


    net = Convolution2D(64, 3, 3, border_mode = 'same')(inp)
    net = Activation(activation)(net)
    net = Convolution2D(64, 3, 3, border_mode = 'same')(net)
    net = Activation(activation)(net)
    net = MaxPooling2D((2, 2), strides = (2, 2))(net)
    net = Dropout(0.6)(net) 
    
    net = Flatten()(net)
    net = Dense(512)(net)
    net = Activation(activation)(net)
    net = Dropout(0.5)(net)
    net = Dense(3)(net)
    net = Activation('softmax')(net)
    
    return inp, net

inp, net = toy2()
model = Model(inp, net)

model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

dirr = './data/train'
classes = ['unidentified', 'green', 'red']

batch_size = 64
nb_epoch = 10

datagen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2, 
                             shear_range=0.05, 
                             zoom_range=.1,
                             fill_mode='nearest', 
                             rescale=1. / 255)
image_data_gen = datagen.flow_from_directory(dirr, 
                                             target_size=(64, 64), 
                                             classes=classes,
                                             batch_size=batch_size)

In [ ]:
model.fit_generator(image_data_gen, nb_epoch=nb_epoch)

In [ ]:
## prediction
from PIL import Image
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

def reshape_image(img):
  img = cv2.resize(img, (64, 64)) 
  x = img_to_array(img)
  return x[None, :]

classes = image_data_gen.class_indices


path = './data/test/'

right = 0
wrong = 0
for i in ['green', 'red', 'unidentified']:
    files = os.listdir(path + i)
    for j in np.arange(len(files)):
        imgpath = path + i + '/' + files[j]
        img = cv2.imread(imgpath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ## plt.imshow(img)
        ## img.shape
        probs = model.predict(reshape_image(img))
        max_prob = probs.argmax(axis = -1)
        predicted_class = list(classes.keys())[list(classes.values()).index(max_prob[0])] 
        
        if predicted_class == i:
            right += 1
        else:
            wrong += 1  
            
print(right)
print(wrong)

In [ ]:
model.save('classifier.h5')